# Inverse dynamics optimization on real data
- This notebook is dedicated to testing the optimization process for inverse dynamics 
- Starting from a rough solution estimated by inverse kinematics
- No consideration of 2 projections is considered here (at least yet!)

In [ ]:
import time 

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pinocchio as pin
from scipy.optimize import least_squares
from scipy.special import huber
from projectyl.utils.properties import ELBOW, SHOULDER, WRIST
from projectyl.utils.arm import plot_optimization_curves

from projectyl.dynamics.inverse_kinematics import build_arm_model
from projectyl.utils.arm import interactive_replay_sequence
from projectyl.utils.io import Dump
%load_ext autoreload
%autoreload 2
global_params = {}
build_arm_model(global_params, headless=False)
arm_robot = global_params["arm"]
viz = global_params["viz"]

### Load data


To load data, you need a preprocessed video file. Assume IK has been performed first.

`python scripts/batch_video_processing.py -i "data/*9_*.mp4" -o __out -A ik --headless`

In [ ]:
data = Dump.load_pickle("../__out/0009_weight_lifting_main_camera_30fps/coarse_ik.pkl")

# INITIAL STATE LIST
q = np.array(data["q"])
T = len(q)

# 3D points
p = ([np.array([el.translation for el in data["3dpoints"][member]]).T for member in [SHOULDER, ELBOW, WRIST]])
gt_p_full = np.concatenate(p).T
gt_p_full.shape #T, 9

# initialization from IK estimation
initial_full = np.concatenate([q, np.zeros((T, 2 * arm_robot.model.nv))], axis=1)
initial_full.shape

In [ ]:
shoulder_frame_id = arm_robot.model.getFrameId(SHOULDER)
elbow_frame_id = arm_robot.model.getFrameId(ELBOW)
wrist_frame_id = arm_robot.model.getFrameId(WRIST)
DT = 1/30. # 30fps

### Define the whole optimizer
> TODO: Move to a python file instead of notebook

In [ ]:
def get_pose_velocity_from_state(tq_normalized, tvq, T):
    # Forward pass
    rec_p = []
    rec_v = []

    for t in range(T):
        q = tq_normalized[t]
        vq = tvq[t]
        # (forward kinematics)
        pin.forwardKinematics(arm_robot.model, arm_robot.data, q, vq)
        pin.updateFramePlacements(arm_robot.model, arm_robot.data)
        # Predicted 3D points -> 3x3 points
        shoulder_p = arm_robot.data.oMf[shoulder_frame_id].translation
        elbow_p = arm_robot.data.oMf[elbow_frame_id].translation
        end_effector_p = arm_robot.data.oMf[wrist_frame_id].translation
        # Predicted 3D velocities -> ingnore shoulder 2x3 points
        shoulder_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, shoulder_frame_id, pin.ReferenceFrame.WORLD).linear
        elbow_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, elbow_frame_id, pin.ReferenceFrame.WORLD).linear
        end_effector_v = pin.getFrameVelocity(arm_robot.model, arm_robot.data, wrist_frame_id, pin.ReferenceFrame.WORLD).linear

        p = np.concatenate([shoulder_p, elbow_p, end_effector_p])
        rec_p.append(p)

        v = np.concatenate([shoulder_v, elbow_v, end_effector_v])
        rec_v.append(v)

    tp = np.vstack(rec_p)
    tv = np.vstack(rec_v)

    ta = (tv[1:] - tv[:-1]) / DT

    return tp, tv, ta

def diff_3D(tp, tp_observed):
    diff = tp - tp_observed
    diff = diff.flatten()

    return diff

def smooth_velocity_acceleration(tv, ta):
    rv = tv.flatten()
    ra = ta.flatten()

    return np.concatenate([rv, ra])

def smooth_torque(ttauq):
    return ttauq.flatten()

# If the Lagrange dynamics formulation cannote be totally satisfied,
# one can relax by minimizing the difference between:
# - the Lagrange dynamics torques tau_rec predicited from q, vq and aq.
# - the current predicted torques ttauq
# question: why not also going backward in time ?
def full_body_dynamics(tq_normalized, tvq, taq, ttauq, T):
    
    tau_rec = []

    for t in range(1, T):
        tau = pin.rnea(arm_robot.model, arm_robot.data, tq_normalized[t], tvq[t], taq[t - 1])
        # taq[t - 1] ?

        tau_rec.append(tau)
    
    ttau = np.vstack(tau_rec)

    diff = ttau - ttauq[1:]
    # ttauq[1:] - cannot compute ttau for t=0
    
    diff = diff.flatten()

    return diff

# Build the cost function
def objective(txuc, tp_observed, T, debug=False) -> np.ndarray:
    txuc_r = txuc.reshape(T, -1)

    tq = txuc_r[:, :arm_robot.model.nq]
    norm_quat = np.linalg.norm(tq[:, :-1], axis=1, keepdims=True)
    tq_normalized = tq.copy()
    tq_normalized[:, :-1] /= norm_quat

    tvq = txuc_r[:, arm_robot.model.nq: arm_robot.model.nq + arm_robot.model.nv]

    taq = (tvq[1:] - tvq[:-1]) / DT

    ttauq = txuc_r[:, arm_robot.model.nq + arm_robot.model.nv:]

    tp, tv, ta = get_pose_velocity_from_state(tq_normalized, tvq, T)

    # DEBUG
    if debug:
        print("3D joint positions", np.linalg.norm(diff_3D(tp, tp_observed)))
        print("Smooth velocity acceleration", np.linalg.norm(smooth_velocity_acceleration(tv, ta)))
        print("Smooth torque", np.linalg.norm(smooth_torque(ttauq)))
        print("Full body dynamics", np.linalg.norm(full_body_dynamics(tq_normalized, tvq, taq, ttauq, T)))

    res = np.concatenate([
        diff_3D(tp, tp_observed),
        0.1 * smooth_velocity_acceleration(tv, ta),
        0.1 * smooth_torque(ttauq),
        10 * full_body_dynamics(tq_normalized, tvq, taq, ttauq, T),
    ])

    return res

### Define trimming
To process just a piece of the sequence

In [ ]:
### DEFINE TRIMMING
trim = 30
assert trim < T
initial= initial_full[:trim]
gt_p = gt_p_full[:trim]
assert gt_p.shape[1] ==9
assert initial.shape[1] ==13

### OPTIMIZE

In [ ]:
sol = least_squares(objective, initial.flatten(), args=(gt_p, trim)).x

### VISUALIZE INTERACTIVELY

In [ ]:
res = sol.reshape(trim, -1)
interactive_replay_sequence(
    {
        "estimated": res[:, :arm_robot.model.nq],
        "groundtruth": initial[:, :arm_robot.model.nq],
    },
    viz
)

### PLOT CURVES

In [ ]:
plot_optimization_curves([
    (initial, "[gt]", "--"),
    ],
    title="Initial guess"
)
plot_optimization_curves([
    (res, "[estimation]", "-"),
    ],
    title="Optimizer prediction"
)

In [ ]:
plot_optimization_curves([
    (res, "[estimation]", "-"),
    ],
    title="Optimizer prediction",
    mode="q"
)